# Spark Tutorial. Part 4 - Advanced ML with MLlib

## Recommendation systems

One of the most common uses of machine learning is a prediction of consumer desires. This allows Google to show you relevant ads, Amazon to recommend relevant products, and Netflix to recommend movies that you might like. This practical lesson will demonstrate you how Apache Spark can be used to implement such a recommendation system. We will start with some basic techniques, and then use the Spark MLlib library's Alternating Least Squares method to make more sophisticated predictions.

In this lab we will analyze a dataset of more than 60.000 purchases stored in the *purchases.csv* file.

*Note: Think carefully before calling `collect()` on RDDs. You should use other ways to do tasks where this is available. Calling `collect()` and then implementing local analysis using Python is the bad practice. It will work only for small datasets. All that can be done with Spark should be done with Spark. Other solutions is likely to fail when grading and will not receive the highest score.*

In [1]:
import os
# export PYSPARK_PYTHON=/usr/bin/python3
# export PYSPARK_DRIVER_PYTHON=ipython3
# export PYSPARK_DRIVER_PYTHON_OPTS="notebook"
# os.environ["PYSPARK_DRIVER_PYTHON"] = "/home/dmitriy/.local/lib/python3.6/site-packages"
os.environ["PYSPARK_PYTHON"]="/usr/bin/python3.6"
# os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook"
# os.environ["SPARK_HOME"] = "/usr/local/spark/spark-2.4.0-bin-hadoop2.7"
# os.environ["SPARK_HOME"] = "/home/dmitriy/Рабочий стол/learn_Spark/spark-2.4.0-bin-hadoop2.7"

In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

In [3]:
# Importing dependencies
import math
from collections import namedtuple
from pyspark.mllib.recommendation import ALS

In this lab we will examine two approaches of creating recommender systems. Therefore, we should create two different RDDs from one main dataset.

Let's start with reading the csv file. Each row of this file contains 3 values:
1. *Buyer* - a GUID (Globally Unique Identifier) of buyer (person who bought a product);
2. *Product* - a string, name of a product (here: cups, plates, glasses and salad bowls);
3. *Purchases* - an integer, quantity of purchased products.

For example, the first row of the dataset looks as follows:

`Y3VzdG9tZXIvNGViNDkwMzI5NGE2NWY5ZDcxMDIyYjM2\tASA Selection 1900013\t55`

You can notice that entries are delimited with tab keys (\t characters). This row means, that buyer with an ID Y3VzdG9tZXIvNGViNDkwMzI5NGE2NWY5ZDcxMDIyYjM2 bought 55 salad bowls named "ASA Selection 1900013".

In [4]:
# Read data and cache it, because it will be used 2 times during this lesson.
raw_data = sc.textFile('purchase.csv', 2).cache()

# Lets print fields of the first row.
print (raw_data.first().replace('\t', ', '))

Y3VzdG9tZXIvNGViNDkwMzI5NGE2NWY5ZDcxMDIyYjM2, Strawberry Cough, 55


In this lab we will examine subsets of the tuples we create. Whenever we examine only a subset of a large dataset, there is a probability that the result will depend on some previous actions, for example, initial order of values or partitioning across the workers. We want to ensure that we are *always getting identical subsets*, regardless of how we manipulate or store the data.

We can achieve that by sorting the dataset before examining it. You might think that the most obvious solution when dealing with an RDD of tuples would be to use the [`sortByKey()`](https://spark.apache.org/docs/1.6.3/api/python/pyspark.html#pyspark.RDD.sortByKey) function. However, this choice is problematic, as we can still end up with different results if the key is not unique.

*Note: It is important to use the [`unicode`](https://docs.python.org/2/howto/unicode.html#the-unicode-type) type instead of the `string` type as the titles may consist of unicode characters.*

Take a look at the following example and note that while the datasets are equal, the sorted sets usually have diferent order of tupels, although they may randomly match up from time to time.

You can try running this multiple times.  If the last assertion fails, don't worry about it: that was just the luck of the draw.  And note that in some environments the results may be more deterministic.

In [5]:
data1 = [(1, u'alpha'), (2, u'alpha'), (2, u'beta'), (3, u'alpha'), (1, u'epsilon'), (1, u'delta')]
data2 = [(1, u'delta'), (2, u'alpha'), (2, u'beta'), (3, u'alpha'), (1, u'epsilon'), (1, u'alpha')]

rdd1 = sc.parallelize(data1)
rdd2 = sc.parallelize(data2)

rdd1_sorted = rdd1.sortByKey().collect()
rdd2_sorted = rdd2.sortByKey().collect()

print (rdd1_sorted)
print (rdd2_sorted)

assert set(rdd1_sorted) == set(rdd2_sorted)     # Note that both lists have the same elements
assert rdd2_sorted[0][0] < rdd2_sorted[-1][0]   # Check that it is sorted by the keys
assert rdd1_sorted[0:2] != rdd2_sorted[0:2]     # Note that the subset consisting of the first two elements does not match

[(1, 'alpha'), (1, 'epsilon'), (1, 'delta'), (2, 'alpha'), (2, 'beta'), (3, 'alpha')]
[(1, 'delta'), (1, 'epsilon'), (1, 'alpha'), (2, 'alpha'), (2, 'beta'), (3, 'alpha')]


Even though the two lists contain identical tuples, the difference in ordering *sometimes* yields a different ordering for the sorted RDD (try running the cell repeatedly and see if the results change or the assertion fails). If we only examined the first two elements of the RDD (e.g., using `take(2)`), then we would observe different answers - *that is a really bad outcome as we want identical input data to always yield identical output*.

A better technique is to sort the RDD by *both the key and value*. We can do this by combining the key and value into a single string and then sorting on that string. But since we know that `tuples` in Python are compared lexicographically, there is no reason to sort on a single string value. Take a look at the following example to understand the idea better.

In [6]:
# Sort by both fields
sort_fn = lambda row: (row[0], row[1])

print (rdd1.sortBy(sort_fn).collect())

# Same as the identity function usage
print (rdd2.sortBy(lambda x: x).collect())

[(1, 'alpha'), (1, 'delta'), (1, 'epsilon'), (2, 'alpha'), (2, 'beta'), (3, 'alpha')]
[(1, 'alpha'), (1, 'delta'), (1, 'epsilon'), (2, 'alpha'), (2, 'beta'), (3, 'alpha')]


When we just want to look at the first few elements of the RDD in sorted order, we can use the [`takeOrdered`](https://spark.apache.org/docs/1.6.3/api/python/pyspark.html#pyspark.RDD.takeOrdered) function without any sorting function (if the rows are already parsed properly).

In [7]:
rdd1_sorted = rdd1.takeOrdered(rdd1.count(), sort_fn)
rdd2_sorted = rdd2.takeOrdered(rdd2.count(), sort_fn)

print (rdd1_sorted)
print (rdd2_sorted)

assert rdd1_sorted == rdd2_sorted

[(1, 'alpha'), (1, 'delta'), (1, 'epsilon'), (2, 'alpha'), (2, 'beta'), (3, 'alpha')]
[(1, 'alpha'), (1, 'delta'), (1, 'epsilon'), (2, 'alpha'), (2, 'beta'), (3, 'alpha')]


Having learned about sorting, we can start data analysis.

## Part 1: Basic Recommendations

One way to recommend products is always recommend those with the highest average number of purchases. In this part, we will use Spark to find the name of a product, number of buyers, and the average number of purchases of the 20 products with the highest average number of purchases and more than 50 buyers. We want to filter our products with high average number of purchases but fewer than or equal to 50 buyers because products with few buyers may not have high demand from other customers.

First, let's create a **`purchases`** RDD from the **`raw_data`** containing tuples of the following structure: `(Product, Purchases)`.

In [8]:
# First, we define a `namedtuple` data structure, that will store entries of the dataset, and name it **`Purchase`**.
Purchase = namedtuple('Purchase', ['buyer', 'product', 'purchases'])

def parse_row(entry):
    data = entry.split('\t')
    return Purchase(data[0], data[1], int(data[2]))

purchases = raw_data.map(parse_row).map(lambda p: (p.product, p.purchases))

print (purchases.take(5))

[('Strawberry Cough', 55), ('Donation to Junju Water Project - default', 2), ('Big Bang 2 Feminized', 10), ('Kingsize Rolling Papers', 5), ('AutoMazar', 9)]


## Exercise 1

Using pure Python, create a function **`compute_counts_and_averages()`** that takes a single tuple of (Product, (Purchases1, Purchases1, Purchases1, ...)) and returns a tuple of (average number of purchases, Product, number of buyers).       
>For instance, given a tuple `(100, (10.0, 20.0, 30.0))`, your function should return `(20.0, 100, 3)`

In [9]:
# Exercise 1

# Implement it
def compute_counts_and_averages(name_and_numbers):
    """ Get average number of purchases
    Args:
        Product_and_AmountsTuple: a single tuple of (Product_Name, (Number1, Number2, Number3, ...))
    Returns:
        tuple: a tuple of (average number of purchases, Product_Name, number of buyers)
    """
    name = name_and_numbers[0]
    numbers = name_and_numbers[1]
    return (
        sum(numbers) / float(len(numbers)),
        name,
        len(numbers))

In [10]:
'''
import sys
sys.path.append("/usr/local/lib/python2.7/site-packages")
sys.path.append('/home/vagrant/.local/lib/python2.7/site-packages')

# load testing library
# from test_helper_spark import *
import os.path
%matplotlib inline
'''

'\nimport sys\nsys.path.append("/usr/local/lib/python2.7/site-packages")\nsys.path.append(\'/home/vagrant/.local/lib/python2.7/site-packages\')\n\n# load testing library\n# from test_helper_spark import *\nimport os.path\n%matplotlib inline\n'

In [11]:
# lab4_test_ex_1(compute_counts_and_averages)

##  Exercise 2

It is possible to compute the average number of purchases, so, we can now use our **`compute_counts_and_averages()`** function with Spark to determine goods with highest average number of purchases.

You should follow this steps:

1. As you can remember, the **`purchases`** contains tuples of the form (Product, Purchases). You should use **`purchases`** to  create new RDD with tuples of the form (Product, Python iterable of Purchases for that product) and save it as **`grouped_purchases`**. You should get the RDD of the following form:
>`[(u'Auto Chemdog', <pyspark.resultiterable.ResultIterable object at 0xb0f204ec>),
(u'Sweet Afghani Delicious Auto', <pyspark.resultiterable.ResultIterable object at 0xb20dc1ec>),      
(u'Race Fuel', <pyspark.resultiterable.ResultIterable object at 0xb20f55ec>)]`.

2. Use **`grouped_purchases`** and **`compute_counts_and_averages()`**  to compute the number of buyers and average number of purchases for each product to get tuples of the form (average number of purchases, Product, number of buyers) and store it as **`average_purchases`**. After this transformation you should obtain an RDD of the following form:
>`[(5.851851851851852, u'Auto Chemdog', 135),
(3.8, u'Sweet Afghani Delicious Auto', 30),
(12.0, u'Race Fuel', 2)]`.

*Hint: you can complete this exercise using only Spark transformations.*

In [12]:
# Exercise 2

# Perform the first step of the exercise
grouped_purchases = purchases.groupByKey()

print ('Grouped purchases: {}\n'.format(grouped_purchases.take(3)))

# Perform the second step of the exercise
average_purchases = grouped_purchases.map(compute_counts_and_averages)

print ('Average purchases: {}'.format(average_purchases.take(3)))

Grouped purchases: [('Strawberry Cough', <pyspark.resultiterable.ResultIterable object at 0x7f0a97e70da0>), ('Big Bang 2 Feminized', <pyspark.resultiterable.ResultIterable object at 0x7f0a97e70128>), ('Kingsize Rolling Papers', <pyspark.resultiterable.ResultIterable object at 0x7f0ac151fc88>)]

Average purchases: [(55.0, 'Strawberry Cough', 1), (1.9968230237338815, 'Big Bang 2 Feminized', 5351), (1.6952380952380952, 'Kingsize Rolling Papers', 105)]


In [13]:
# Test
# lab4_test_ex_2(grouped_purchases, average_purchases)

## Exercise 3

Since we have an RDD of the goods with average number of purchases, so, we can use Spark to determine goods with highest average number of purchases with more than 50 buyers.

1. Perform a single transformation of the RDD on the **`average_purchases`** to get goods with more than 50 buyers.
2. Then use the **`sort_fn()`** function (implemented earlier in this lab) to sort values by the average number of purchases to get the goods in order of their mean of purchases (highest average number of purchases first) and save it as **`valuable_purchases`**. The result should be an RDD of the form:
>`[(9.636363636363637, u'Cluster Bomb Feminised', 55),
(9.054054054054054, u'Bubblelicious', 74),
(8.771186440677965, u'Aurora Indica', 118)]`
3. Extract top 20 names of goods and save them as **`recommended_goods`**.

In [14]:
# Exercise 3

# Apply an RDD transformation to `average_purchases` to limit the results to products with
# number of purchases from more than 50 buyers. We then use the `comparator()` helper function to sort by the
# average number of purchases to get the products in order of their average number of purchases (descending).
valuable_purchases = (average_purchases
                      .filter(lambda p: p[2] > 50)
                      .sortBy(sort_fn, False))

recommended_products = valuable_purchases.map(lambda p: p[1]).take(20)

print ('Goods with the highest average number of purchases: {}'.format(recommended_products))

Goods with the highest average number of purchases: ['AK47', 'Berry Bomb', 'Ata Tundra', 'Skunk #1', 'Pure Power Plant', 'Cluster Bomb Feminised', 'Bubblelicious', 'Aurora Indica', 'GreenHouse Seeds White Rhino Feminized', 'Big Buddha Critical Mass Automatic Feminized', 'GreenHouse Seeds Cheese Feminized', 'AK 48', 'Greenhouse Seeds Exodus Cheese Feminized', 'Auto NYC Diesel', 'Big Buddha Cheese Feminized', 'THC Bomb Feminised', 'Big Buddha Blue Cheese feminized', 'Auto Mix', 'Skywalker Kush', 'GreenHouse Seeds Big Bang Feminized']


In [15]:
# Test
# lab4_test_ex_3(recommended_products)

So, we got the top 20 products that can be recommended to users. While using a threshold on the number of buyers is one way to improve the recommendations, there are many better ways to improve quality of a recommender system.

## Part 2: Collaborative Filtering

In this part, you will learn how to use MLlib to make personalized recommendations using the goods data we analyzed earlier.

We are going to use a technique called [collaborative filtering](https://en.wikipedia.org/wiki/Collaborative_filtering). Collaborative filtering allows to make automatic predictions (filtering) about the preferences of a person by collecting taste information from many users (collaborating). The basic assumption of this technique is that if some person has the same opinion as some other person on an issue, it is more likely that these people have the same opinion on a different issue.

## Exercise 4

In this part, before creating **`purchases`**, we should build two dictionaries. We have to do this as a preparation for the further analysis.

The first dictionary will map **buyer keys** (represented by unicode strings) to unique numbers. The second dictionary will be created for the same purpouse, but it will store **product names** as a keys.

So, the **`indexed_buyers`** dictionary, that you will create, should have the following structure:
```
[(u'Y3VzdG9tZXIvNTUzNjg4MzRkMDM3NzJiYTQ2OTE5YjBk', 0),
 (u'Y3VzdG9tZXIvNTQ1MzkyMjFkMDM3NzIwZTJlOGI1NmQy', 1),
 (u'Y3VzdG9tZXIvNTQ4NGZkZmZkMDM3NzJiZjFiOGM0MjE4', 2), ...]
```

*Hint: use [`zipWithIndex()`](http://spark.apache.org/docs/1.6.3/api/python/pyspark.html#pyspark.RDD.zipWithIndex) to add index to each value in dataset.*

In [16]:
# Exercise 4.1

# Implement a function that accepts a row from dataset
# and returns a buyer id as a unicode string
def parse_buyer_id(entry):
    return entry.split('\t')[0]

indexed_buyers = dict(
    raw_data
        .map(parse_buyer_id)
        .distinct()
        .zipWithIndex()
        .collect()
)

print (len(indexed_buyers))

19561


Now perform same operations to build the **`indexed_products`** dictionary.

In [17]:
# Exercise 4.2

# Implement a function that accepts a row from dataset
# and returns a product name as a unicode string
def parse_product_name(entry):
    return entry.split('\t')[1]

indexed_products = dict(
    raw_data
        .map(parse_product_name)
        .distinct()
        .zipWithIndex()
        .collect()
)

print (len(indexed_products))

740


In [18]:
# Test
# lab4_test_ex_4_1(indexed_buyers, indexed_products)

Since we have built two dictionaries, we can replace string values in the original dataset with the corresponding numerical indexes.

Implement a funciton that converts entries from the dataset to tuples of three numerical values. For example,  
```
[u'Y3VzdG9tZXIvNGViNDkwMzI5NGE2NWY5ZDcxMDIyYjM2\tStrawberry Cough\t55',
 u'Y3VzdG9tZXIvNTQ1MzkyMjFkMDM3NzIwZTJlOGI1NmQy\tDonation to Junju Water Project - default\t2',    
 u'Y3VzdG9tZXIvNTQ4NGZkZmZkMDM3NzJiZjFiOGM0MjE4\tBig Bang 2 Feminized\t10']
```
will be transformed to
```
[(0, 0, 55)',
 (1, 1, 2)',    
 (2, 2, 10)']
```

In [19]:
# Exercise 4.3

# Using the indexed_buyers and indexed_products dicts, split an entry from the dataset
# to a tuple, containing 3 integer values: (buyer_idx, product_idx, purchases)
def parse_entry(entry):
    items = entry.split('\t')
    return (
        indexed_buyers[items[0]],
        indexed_products[items[1]],
        int(items[2])
    )

purchases = raw_data.map(parse_entry)
print (purchases.take(5))

[(0, 0, 55), (0, 361, 2), (0, 1, 10), (0, 2, 5), (0, 362, 9)]


In [20]:
# Test
# lab4_test_ex_4_3(purchases)

Before we start training our models, we need to break up the **`purchases`** dataset into 3 parts: train, validation and test sets. We will use the first one directly to train our models. The second will serve as a dataset for picking the best model and tuning hyperparameters. The test dataset will be used for measuring the performance of the model.

*Hint: Use the [randomSplit()](https://spark.apache.org/docs/1.6.3/api/python/pyspark.html#pyspark.RDD.randomSplit) transformation to split one rdd into multiple rdd's.*

In [21]:
# Split the dataset into 3 subsets:
train_data, val_data, test_data = purchases.randomSplit([0.75, 0.15, 0.1], 42)

# Take a look at the exaamples from each subset
print (train_data.take(3))
print (val_data.take(3))
print (test_data.take(3))

# Count how many items are in each subset
print ('We have %s training examples' % train_data.count())
print ('We have %s validation examples' % val_data.count())
print ('We have %s testing examples' % test_data.count())

# Check if everything is correct 
# assert train_data.count() == 47403
# assert val_data.count() == 9590
# assert test_data.count() == 6333

[(0, 0, 55), (0, 361, 2), (0, 1, 10)]
[(0, 364, 5), (0, 366, 10), (9800, 8, 8)]
[(9800, 2, 48), (9800, 3, 6), (9800, 376, 10)]
We have 47537 training examples
We have 9402 validation examples
We have 6387 testing examples


You can see, that after performing the split, we got approximately 47,5 thousand training examples, about 9,5 validation examples, and about 6,3 examples from the test set. It is possible that the exact numbers vary a little bit due to some randomness in the `randomSplit()` function.

# Root Mean Squared Error (RMSE)
When you will build different machine learning algorithms, you will need to evaluate each model to compare  them and decide which models are better. So, it will be good to have single indicator of the model's performance. In the data science there are several approaches to the evaluation of the model. One of them is the [Root Mean Square Error](https://en.wikipedia.org/wiki/Root-mean-square_deviation) (RMSE) or Root Mean Square Deviation (RMSD). This metric allows to measure the error of the model. You can use RMSE if you have the actual (true) answers and the answers predicted by your model. This indicator represents the sample standard deviation of the differences between predicted values and observed values. RMSE computed on the train set can be called as train error while the RMSE of the test set will serve as the test error. Large RMSE means low predictive power of the model and vice verca. You should take into account the fact that RMSE is a scale-dependent metric. This means, that it can be used for the comparation of different models only if the target variable has the same scale. For example, if you predict temperature using the Celsius system, then you can compare all models that generate predictions in Celsius too, but you can not compare these models with the models which generates predictions of the temperatures in the Fahrenheit system. 
The RMSE is the square root of the average value of the square of `(actual number - predicted number)` for all clients and products for which we have the actual value. 

Versions of Spark 1.4 and later has build-in MLlib module called a [RegressionMetrics](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.evaluation.RegressionMetrics). This module can be used to compute RMSE. Nevertheless, we are using an earlier version of Spark, so, we need to write own function to compute RMSE. 

## Exercise 5
> You should create a function which will compute RMSE given `predRDD` and `trueRDD` RDDs. These RDDs consist of tuples of the form (BuyerID, Product_Name, Number).  
Given two RDDs, *X* and *Y* of size *n*, we define RSME as follows: $ RMSE = \sqrt{\frac{\sum_{i = 1}^{n} (X_i - Y_i)^2}{n}}$      
RMSE can be calculated using the following steps:       
* Transform `predRDD` into the tuples of the form ((BuyerID, Product_Name), Number).       
* Transform `trueRDD` into the tuples of the form ((BuyerID, Product_Name), Number).     
* Compute the squared error for each entry where the pair (BuyerID, Product_name) coincide in both RDDs. You should remember, that not every pair appears in both RDDs and therefore these entries will not participate in the calculations. As a result, you should get an RDD with entries of the form $ (X_i - Y_i)^2$. You can use Python's [math](https://docs.python.org/2/library/math.html) module in calculations.       
* Compute the total squared error: $ SE = \sum_{i = 1}^{n} (X_i - Y_i)^2 $.       
* Compute *n* - the number of pairs which were using to calculate the squared error.     
* Use two numbers which you have calculated above to perform final calculations and derive RMSE. The type of output number should be [float](https://docs.python.org/2/library/stdtypes.html#numeric-types-int-float-long-complex).

#### Note: Your solution must only use transformations and actions on RDDs. Do _not_ call `collect()` on either RDD.

In [22]:
# Exercise 5

import math

def get_RMSE(predRDD, trueRDD):
    """ Compute the RMSE between the input's RDDs
    Args:
        predRDD: predicted number of purchases for each buyer and product where each entry is in the form
                      (BuyerID, Product_Name, Number)
        trueRDD: actual value of purchases where each entry is in the form (BuyerID, Product_Name, Number)
    Returns:
        RSME (float): computed root mean squared error
    """
    # Step 1
    predTransfRDD = predRDD.map(lambda row: ((row[0], row[1]), row[2]))

    # Step 2
    trueTransfRDD = trueRDD.map(lambda row: ((row[0], row[1]), row[2]))

    # Step 3
    # Hint: here you should think how to join RDDs to perform map() function on the joined RDD
    squared_errorsRDD = (predTransfRDD
                        .join(trueTransfRDD)
                        .map(lambda row: (row[1][0] - row[1][1])**2))

    # Step 4
    totalError = squared_errorsRDD.reduce(lambda a,b: a+b)

    # Step 5
    numRatings = squared_errorsRDD.count()

    # Step 6
    return math.sqrt(totalError/float(numRatings))


# Check yourself:
y_pred = sc.parallelize([
     (10, 8, 10),
     (4, 1, 3),
     (3, 1, 4),
     (5, 5, 3)])

y_true = sc.parallelize([
     (5, 8, 7),
     (3, 1, 2),
     (5, 3, 5),
     (4, 5, 6),
     (1, 2, 3),
     (2, 2, 2),
     (5, 5, 4)])
        
RMSE_check = get_RMSE(y_pred, y_true)
print ('Error for test dataset (should be 1.58113883008): %s' % RMSE_check)

Error for test dataset (should be 1.58113883008): 1.5811388300841898


In [23]:
# Test

y_pred1 = sc.parallelize([
    (2, 2, 2),
    (1, 2, 5),
    (1, 3, 4),
    (5, 3, 5),
    (2, 2, 2),
    (5, 5, 6),
    (9, 8, 7)])

y_pred2 = sc.parallelize([
     (5, 8, 10),
     (4, 5, 3),
     (3, 1, 4)])

RMSE1 = get_RMSE(y_pred1, y_true)
RMSE2 = get_RMSE(y_pred2, y_true)
RMSE3 = get_RMSE(y_true, y_true)

error_list = [RMSE1, RMSE2, RMSE3]
print(error_list)

[1.2649110640673518, 2.70801280154532, 0.0]


In [24]:
# lab4_test_ex_5(error_list)

# Alternating Least Squares

Alternating Least Squares is one of the algorithms for recommendation systems. You can learn more about this model [here](http://danielnee.com/2016/09/collaborative-filtering-using-alternating-least-squares/). Your task is to implement the ALS algorithm for our case. Spark allows you to train ALS by using build-in function: [ALS.train()](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS). There are several parameters to tune in order to get the best possible results. You will try different hyperparameters and use the best parameters in the rest of the exercises.

## Exercise 6
> Follow these steps to find the best model:       
* Select the necessary hyperparameters of the model. You should pay attention to the parameter *rank*, which is very important. The collaborative filtering is the algorithm which allows to "learn" all needed features. In our case, the features is the preferences of the buyers and the characteristics of the products. When we know these values, we can simply multiply corresponding numbers to compute the amount of purchase. So, our task is to learn 2 parameters vectors (buyers' preferences and products characteristics) by using the already known amounts of purchases. The ALS algorithm fill both these vectors by random values, and then iteratively optimize these parameters vectors until the error on the known examples will be minimal. The hyperparameter *rank* is the length of the parameters vectors. You should understand, that high rank may cause overfitting (high variance, high error on the test set and low error on the training dataset), and the lower rank may cause underfitting (high bias, or high error on the training dataset). In this exercises, you should try to train your models using 2, 4, 8, 12, 16 ranks.       
* Now you can train your model. Use `seed = 42`. Parameter`iterations` should be 20. Also, it is recommended to tune regularization paramter lambda. Try lambda 0.1, 0.5 and 0.9.      
* You should transform your `val_data` to get the values without true answers. Leave only (BuyerID, Product_Name) part of the val_data. At the end, you should get an RDD of the form: `[(15070, 414), (15070, 497), (858, 497)]`.       
* Call [.predictAll](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.predictAll) method to make predictions. The output will be in the format (BuyerID, Product_Name, Number). 
* Measure the performance of your models by using the `get_RMSE` function. Use the predictions of your models and the `val_data` as the input.   
Now you can detect which parameters are optimal for this situation.
#### Note: It is likely that this operation will take a noticeable amount of time; you can observe its progress on the [Spark Web UI](http://localhost:4040). Probably most of the time will be spent running your `get_RMSE()` function, since, unlike the Spark ALS implementation (and the Spark 1.4 [RegressionMetrics](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.evaluation.RegressionMetrics) module), this does not use a fast linear algebra library and needs to run some Python code for all 100k entries. You can take a coffee while Spark will train models. 

In [25]:
'''
def calc(*args):
    return (args[0],args[1])
list2 = val_data.map(lambda x: (x[0], x[1]))
print(list2.take(10))
'''

'\ndef calc(*args):\n    return (args[0],args[1])\nlist2 = val_data.map(lambda x: (x[0], x[1]))\nprint(list2.take(10))\n'

In [26]:
from pyspark.mllib.recommendation import ALS
from pyspark import StorageLevel
from itertools import starmap

val_data_predict = val_data.map(lambda x: (x[0], x[1]))#.persist(StorageLevel(True, True, False, True, 1))

seed = 42
iterations = 1
lambda_list = [0.1, 0.5, 0.9]
rank_list = [2, 4, 8, 12, 16]
errors = []

min_rmse = float('inf')
optimal_rank = -1
optimal_lambda = -1

for rank in rank_list:
    for lambda_ in lambda_list:
        model = ALS.train(train_data, rank, seed=seed, iterations=iterations, lambda_=lambda_)
        predictions = model.predictAll(val_data_predict)
        RMSE = get_RMSE(predictions, val_data)
        errors.append(RMSE)
        print ('Rank %s and lambda %s produces the RMSE: %s' % (rank, lambda_, RMSE))
        if RMSE < min_rmse:
            min_rmse = RMSE
            optimal_rank = rank
            optimal_lambda = lambda_

print ('The best model was trained with rank %s and lambda %s' % (optimal_rank, optimal_lambda))


Rank 2 and lambda 0.1 produces the RMSE: 7.367151741752148
Rank 2 and lambda 0.5 produces the RMSE: 5.467439011619866
Rank 2 and lambda 0.9 produces the RMSE: 5.257034490728869
Rank 4 and lambda 0.1 produces the RMSE: 7.441659015766718
Rank 4 and lambda 0.5 produces the RMSE: 5.367764848581356
Rank 4 and lambda 0.9 produces the RMSE: 5.204473668374107
Rank 8 and lambda 0.1 produces the RMSE: 6.760411764491131
Rank 8 and lambda 0.5 produces the RMSE: 5.330104000060196
Rank 8 and lambda 0.9 produces the RMSE: 5.189414004400124
Rank 12 and lambda 0.1 produces the RMSE: 6.163346813530342
Rank 12 and lambda 0.5 produces the RMSE: 5.215809252247815
Rank 12 and lambda 0.9 produces the RMSE: 5.157735393918228
Rank 16 and lambda 0.1 produces the RMSE: 5.891489956902845
Rank 16 and lambda 0.5 produces the RMSE: 5.219485799441604
Rank 16 and lambda 0.9 produces the RMSE: 5.161152993511684
The best model was trained with rank 12 and lambda 0.9


In [27]:
# Test

# lab4_test_ex_6(errors)

# Test the model

We selected the best hyperparameters of ALS model by using training and validation datasets. But we still is vulnerable to overfitting, because our hyperparameters may fit well only for validation set but the performance would be poor in the real-world. So, we need to check the generalizable ability of our model. We will use `test_data` to test the model with the selected `optimal_rank` and `optimal_lambda`. 

## Exercise 7
> You should follow the next steps in this assignment: 
*  Train your ALS model using the optimal parameters from the previous task: `optimal_rank`, `optimal_lambda` and the `train_data`. All other parameters remains the same.
*  Using the `test_data`, create the RDD of the form (BuyerID, Product_Name) to use it as an input into the `.predictAll` method.
*  Use [.predictAll()](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.predictAll) method to get the predictions for the test dataset.
*  Compute the error using the `get_RMSE` function. Actual labels are contained in the `test_data`. 

In [28]:
seed = 42
iterations = 1
# regularization = optimal_lambda
regularization = 0.4
# rank = optimal_rank
rank = 9
optimal_model = ALS.train(train_data, rank, seed=seed, iterations=iterations, lambda_=regularization)
test_data_predict = test_data.map(lambda x:(x[0],x[1]))
predictions = optimal_model.predictAll(test_data_predict)
RMSE_test = get_RMSE(predictions, test_data)

print ('When using the optimal parameters we can get the following RMSE on the test set: %s' % RMSE_test)

When using the optimal parameters we can get the following RMSE on the test set: 5.676846213285859


In [29]:
# Test

# lab4_test_ex_7(RMSE_test, predictions)

# Compare your model with the base model

You have built your ALS model which is able to predict the amount of purchase of a certain product made by a certain buyer. It is quite complex model, but you should understand, that there are many different models and some of them are very simple. People often use these simple models as the base models, to compare the results of complex model and simple models. Let's make such a  comparison. We will compute the average purchase amount using the training set, and this value will be our prediction for all datapoints in the test set. Then, we will compute the error of this simple model to compare it with ALS model.

## Exercise 8
> Follow these steps to complete the assignment:
* Compute the average purchase amount using all the products in the `train_data`.
* Create new RDD using `test_data` and the average purchase amount. You predict the average purchase amount for each entry in the testing dataset.
* Calculate the error by passing your predictions and the `test_data` in the `get_RMSE` function.

In [30]:
train_avg_purchase = float(train_data.map(lambda x: x[2]).sum()) / train_data.count()
print ('The average purchase amount in the training dataset is %s' % train_avg_purchase)

test_data_with_avgs = test_data.map(lambda x: (x[0], x[1], train_avg_purchase))
RMSE_test_with_avgs = get_RMSE(test_data_with_avgs, test_data)
print ('RMSE on the test dataset with average purchase in the train set as prediction is %s' % RMSE_test_with_avgs)

The average purchase amount in the training dataset is 3.8371584239644907
RMSE on the test dataset with average purchase in the train set as prediction is 4.02978777375156


In [31]:
# Test

# lab4_test_ex_8(train_avg_purchase, RMSE_test_with_avgs)

<center><h3>Presented by <a target="_blank" rel="noopener noreferrer nofollow" href="http://datascience-school.com">datascience-school.com</a></h3></center>